In [ ]:
import torch, detectron2
# import tqdm as notebook_tqdm
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from matplotlib import pyplot as plt


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import load_lvis_json,get_lvis_instances_meta, register_lvis_instances

In [ ]:
def clip_or_pad_to_fixed_size_pytorch(input_tensor, size, constant_values=0):
  """Pads data to a fixed length at the first dimension.
  Args:
    input_tensor: `Tensor` with any dimension.
    size: `int` number for the first dimension of output Tensor.
    constant_values: `int` value assigned to the paddings.
  Returns:
    `Tensor` with the first dimension padded to `size`.
  """
  input_shape = list(input_tensor.shape)
  # print(input_shape)
  # padding_shape = []
  
  input_length = input_shape[0]
  input_length = min(input_length,size)
  input_length = max(0,input_length)

  input_tensor = input_tensor[:input_length]
  # print(input_tensor)

  padding_length = max(0, size - input_length)
  # print (padding_length)
  # padding_shape.append(padding_length)

  # for i in range(1, len(input_shape)):
  #   padding_shape.append(tf.shape(input_tensor)[i])


  # Pads input tensor to the fixed first dimension.
  
  if len(input_tensor.shape[1:]) > 0:
    paddings_size = tuple([padding_length]+ list(input_tensor.shape)[1:])
    paddings = constant_values * torch.ones(paddings_size)
  else:
    paddings = constant_values * torch.ones((padding_length))
    
  # print(input_tensor.shape,paddings.shape)
  # print(paddings.shape,input_tensor.shape)
  # print(torch.concat((input_tensor, paddings), axis=0))
  # print()
  if len(input_tensor.shape[1:]) > 0:
    total_size = tuple([size]+ list(input_tensor.shape)[1:])
    padded_tensor = torch.concat([input_tensor, paddings], axis=0).reshape(total_size)
  else:
    padded_tensor = torch.concat([input_tensor, paddings], axis=0).reshape(size)
  # padded_tensor = torch.concat([input_tensor, paddings], axis=0).reshape(size, input_tensor.shape[1:])
  # output_shape = input_shape
  # output_shape[0] = size
  # padded_tensor.set_shape(output_shape)
  return padded_tensor

In [ ]:
import copy


# 
def get_dict():
    d = load_lvis_json("lvis_v1_small_cocofied.json","")
    # len of segmentation
    _max_element_in_segmentation = 1500
    # num of segmentation
    _max_segmentation = 50
    for i in range(len(d)): 
        for j in range(len(d[i]['annotations'])):
            # make len of segmentation the same
            # num of segmentation in a anno
            for k in range(len(d[i]['annotations'][j]['segmentation'])):
                # print(i,j,k)
                segmentation = torch.tensor(copy.deepcopy(d[i]['annotations'][j]['segmentation'][k]))
                # print(segmentation.shape)
                new_segmentation = clip_or_pad_to_fixed_size_pytorch(segmentation,_max_element_in_segmentation,-1)
                d[i]['annotations'][j]['segmentation'][k] = torch.round(new_segmentation,decimals=2).tolist()      
            # padding the segmentation
            segmentation_set = torch.tensor(copy.deepcopy(d[i]['annotations'][j]['segmentation']))
            new_segmentation_set = clip_or_pad_to_fixed_size_pytorch(segmentation_set,_max_segmentation,-1)
            d[i]['annotations'][j]['segmentation'] = torch.round(new_segmentation_set,decimals=2).tolist()
    return d

In [ ]:
from detectron2.data import MetadataCatalog, DatasetCatalog

DatasetCatalog.register("lvis_v1_small_cocofied_padded", lambda : get_dict())